In [1]:
from datetime import datetime,timedelta
import os
import pandas as pd
import psycopg2
import pickle
from exceptions.databasealredyfilled import DatabaseAlreadyFilledException
from getrootdirectory import getRootDirectory
from configuration.parameters import ConfigurationParameters

In [26]:

from datetime import datetime,timedelta
import os
import pandas as pd
import psycopg2
import pickle
from exceptions.databasealredyfilled import DatabaseAlreadyFilledException
from getrootdirectory import getRootDirectory
from configuration.parameters import ConfigurationParameters

class GTFSCurrentDayAnalysis:
    def __init__(self,filePath:str) -> None:
        self.parentDirectory=os.getcwd()
        self.configurationParameters=ConfigurationParameters()
        
        self.provider="vvs"
    def getTodaysServices(self):
        date_parser = lambda x: pd.to_datetime(x, format='%Y%m%d')
        dayOfWeek=datetime.now().strftime('%A')
        dayOfWeek=dayOfWeek.lower()
        calendar_formatted = pd.read_csv(os.path.join(os.getcwd(),"data",self.provider,"gtfs","calendar.txt"), parse_dates=['start_date','end_date'], date_parser=date_parser)
        todaysServices=calendar_formatted.loc[((calendar_formatted["start_date"]<date_parser(datetime.today().strftime('%Y%m%d'))) & (calendar_formatted["end_date"]>date_parser(datetime.today().strftime('%Y%m%d'))) & (calendar_formatted[dayOfWeek]==1))]
        calendar_dates=pd.read_csv(os.path.join(os.getcwd(),"data",self.provider,"gtfs","calendar_dates.txt"),parse_dates=['date'], date_parser=date_parser)
        
        includedsServicesServices=calendar_dates.loc[(calendar_dates["date"]==date_parser(datetime.today().strftime('%Y%m%d'))) & (calendar_dates["exception_type"]==1)]
    

        excludedServvices=calendar_dates.loc[(calendar_dates["date"]==date_parser(datetime.today().strftime('%Y%m%d'))) & (calendar_dates["exception_type"]==2)]
        originalServices=set(list(todaysServices["service_id"]))
        removedServices=set(list(excludedServvices["service_id"]))
        addedServices=set(list(includedsServicesServices["service_id"]))
        activeServices=originalServices.difference(removedServices)
        activeServices=activeServices.union(addedServices)
        todaysServices=pd.DataFrame(columns=["service_id","date"])
        todaysServices["service_id"]=list(activeServices)
        todaysServices["date"]=[datetime.today().strftime("%Y%m%d")]*len(activeServices)
        
        return todaysServices
    
    def getTodaysTrips(self):
        todaysServices=self.getTodaysServices()
        trips=pd.read_csv(os.path.join(self.parentDirectory,"data",self.provider,"gtfs","trips.txt"))
        todayTrips=pd.merge(trips, todaysServices, on='service_id', how='inner')
        todayUniqueTripds=todayTrips.drop_duplicates(subset=["trip_id"])
        
        #fetch the agency id's from routs.txt
        
        routes=pd.read_csv(os.path.join(getRootDirectory(),"data",self.provider,"gtfs","routes.txt"))
        todaysTripsWithRoutes=pd.merge(todayUniqueTripds,routes,on="route_id",how="inner")
        todaysTripsWithRoutesSubset=todaysTripsWithRoutes[['route_id', 'trip_id', 'service_id','agency_id','route_long_name', 'trip_headsign', 'direction_id',
            'shape_id',  'date']]
        return todaysTripsWithRoutesSubset
    
    def generateTimeStamp(self,day,time:str):
        try:
            timestampStr=day+" "+time
            correct_timestamp=datetime.strptime(timestampStr,"%Y%m%d %H:%M:%S")
        except:
            
            timeparts=time.split(":")
            hours=(timeparts[0])
            minutes=int(timeparts[1])
            correctedHours=correctedHours=int(hours)-24
            current_datetime = datetime.today()

                # Set the time to midnight (00:00:00)
            beginning_of_tomorrow = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)

            correct_timestamp=beginning_of_tomorrow+timedelta(days=1,hours=correctedHours,minutes=minutes)
                
            
        return correct_timestamp
    
    def getDeparturesAndArrivalsToday(self):
        todaysTrips=self.getTodaysTrips()
        dtype = {"date": str}
        stopTimes=pd.read_csv(os.path.join(self.parentDirectory,"data",self.provider,"gtfs","stop_times.txt"),dtype=dtype)
        first_stop = stopTimes['stop_sequence'] == 1
        last_stop = stopTimes.groupby('trip_id')['stop_sequence'].transform('max') == stopTimes['stop_sequence']
        subset = stopTimes[first_stop | last_stop]
        subset
        
        todaysDepartureArrivals=pd.merge(subset,todaysTrips,on="trip_id",how="inner")
        todaysDepartureArrivals.sort_values(by=['trip_id', 'stop_sequence'])
        todaysDepartureArrivals.to_csv(os.path.join(self.parentDirectory,"data",self.provider,datetime.now().strftime("%Y-%m-%d")+".txt"))
        
        timestamps=[]
        agencies=[]
        agenciesFlat=[]
        trip_ids=[]
        beginTimes=[]
        endTimes=[]
        origins=[]
        destinations=[]
        
        longNames=[]
        
        for i in range(len(todaysDepartureArrivals)):
            row=todaysDepartureArrivals.iloc[i]
            day=row["date"]
            currentTripId=row["trip_id"]
            
            
            if(row["stop_sequence"]==1):
                time=row["departure_time"]
                timestamp=self.generateTimeStamp(day,time)
                beginTimes.append(timestamp)
                timestamps.append(timestamp)
                trip_ids.append(currentTripId)
                agency=row["agency_id"]
                
                agencies.append(agency)
                agenciesFlat.append(agency)
                longname=row["route_long_name"]
                longNames.append(longname)
                
                originDhid=row["stop_id"]
                origins.append(originDhid)
                
                
                
                
                        
                
            else:
                    
                
                time=row["arrival_time"]
                agency=row["agency_id"]
                agencies.append(agency)
                timestamp=self.generateTimeStamp(day,time)
                endTimes.append(timestamp)
                timestamps.append(timestamp)
                
                destinationDhid=row["stop_id"]
                destinations.append(destinationDhid)
                
                
                
              
                
            
            
            
           
                    
            
        todaysDepartureArrivals["timestamp"]=timestamps
        todaysDepartureArrivals["agency"]=agencies
        
       
        
        flatDataset=pd.DataFrame(columns=["trip_id","begin_time","end_time","agency"])
        flatDataset["trip_id"]=trip_ids
        flatDataset["begin_time"]=beginTimes
        flatDataset["end_time"]=endTimes
        flatDataset["agency"]=agenciesFlat
        flatDataset["route_long_name"]=longNames
        flatDataset["number of updates"]=[0]*len(trip_ids)
        flatDataset["origin"]=origins
        flatDataset["destination"]=destinations
        print("number of origins "+str(len(origins)))
        print("number of destinations "+str(len(destinations)))
        flatDataset.to_pickle(os.path.join(self.parentDirectory,"data",self.provider,"flat"+datetime.now().strftime("%Y-%m-%d")+".pkl"))
        #flatDataset.to_pickle("today_flat"+".pkl")
        flatDataset.to_csv(os.path.join(self.parentDirectory,"data",self.provider,"flat"+datetime.now().strftime("%Y-%m-%d")+".txt"))
        
        todaySubset= todaysDepartureArrivals[["trip_id","arrival_time", "departure_time", "stop_id","stop_sequence",
                                        "stop_headsign","route_id", "service_id","trip_headsign","direction_id",
                                        "date", "timestamp", "agency"]]
        todaySubset["minute of day"]=todaySubset['timestamp'].dt.hour * 60 + todaySubset['timestamp'].dt.minute
        todaySubset.to_pickle(os.path.join(self.parentDirectory,"data",self.provider,datetime.now().strftime("%Y-%m-%d")+".pkl"))
        return flatDataset

In [27]:
import os
print(os.getcwd())
mycurrentTag=GTFSCurrentDayAnalysis("somestfu")

todayTrips=mycurrentTag.getDeparturesAndArrivalsToday()

c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator


C:\Users\CHO\AppData\Local\Temp\ipykernel_10112\682543815.py:20: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  calendar_formatted = pd.read_csv(os.path.join(os.getcwd(),"data",self.provider,"gtfs","calendar.txt"), parse_dates=['start_date','end_date'], date_parser=date_parser)
C:\Users\CHO\AppData\Local\Temp\ipykernel_10112\682543815.py:22: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  calendar_dates=pd.read_csv(os.path.join(os.getcwd(),"data",self.provider,"gtfs","calendar_dates.txt"),parse_dates=['date'], date_parser=date_parser)


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator


C:\Users\CHO\AppData\Local\Temp\ipykernel_10112\682543815.py:76: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  stopTimes=pd.read_csv(os.path.join(self.parentDirectory,"data",self.provider,"gtfs","stop_times.txt"),dtype=dtype)


number of origins 31298
number of destinations 31298


C:\Users\CHO\AppData\Local\Temp\ipykernel_10112\682543815.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todaySubset["minute of day"]=todaySubset['timestamp'].dt.hour * 60 + todaySubset['timestamp'].dt.minute


In [5]:
todayTrips.head()

,trip_id,begin_time,end_time,agency,route_long_name,number of updates,origin,destination
0,de:vvs:10001_::1-T0-100-j24,2024-11-06 17:04:00,2024-11-06 18:07:00,VVS,Kirchheim (T) - Plochingen - Stuttgart - Herre...,0,de:08115:4512:1:2,de:08116:7802:1:10
1,de:vvs:10001_::1-T0-101-j24,2024-11-06 04:23:00,2024-11-06 04:50:00,VVS,Kirchheim (T) - Plochingen - Stuttgart - Herre...,0,de:08116:7802:1:9,de:08111:6115:2:4
2,de:vvs:10001_::1-T0-102-j24,2024-11-06 17:31:00,2024-11-06 18:37:00,VVS,Kirchheim (T) - Plochingen - Stuttgart - Herre...,0,de:08115:4512:1:2,de:08116:7802:1:10
3,de:vvs:10001_::1-T0-103-j24,2024-11-06 17:46:00,2024-11-06 19:09:00,VVS,Kirchheim (T) - Plochingen - Stuttgart - Herre...,0,de:08115:4512:1:2,de:08116:4211:1:2
4,de:vvs:10001_::1-T0-104-j24,2024-11-06 17:16:00,2024-11-06 18:39:00,VVS,Kirchheim (T) - Plochingen - Stuttgart - Herre...,0,de:08115:4512:1:2,de:08116:4211:1:2


In [ ]:
class TestClass:
    def __init__(self,firstName,middleName,lastName) -> None:
        self.firstName=firstName
        self.middleName=middleName
        self.lastName=lastName
    def __eq__(self, other: object) -> bool:
        if isinstance(other,TestClass):
            print("instance test ")
            if((self.firstName==other.firstName) and (self.lastName==other.lastName)):
                return True
            else:
                return False
        else:
            return False
    def __hash__(self):
        return hash((self.firstName, self.middleName, self.lastName))
            
          

In [37]:
a=TestClass("Sachin","Dev","Tendulkar")
b=TestClass("Sachin","Dev","Tendulkar")
c=TestClass("Ajay","J","Jadega")
d=TestClass("Andrew","M","Simmons")


A={a,c}
B={b,c,d}

In [19]:
class EfaTrip:
    def __init__(self,beginTime,endTime,origin,destination,parentTripId) -> None:
        self.beginTime=beginTime
        self.endTime=endTime
        self.origin=origin
        self.destination=destination
        self.parentTripId=parentTripId
        
    def __eq__(self, other: object) -> bool:
        if isinstance(other,EfaTrip):
            print("instance test ")
            if((self.beginTime==other.beginTime) and (self.endTime==other.endTime) and (self.origin==other.origin) and (self.destination==other.destination)):
                return True
            else:
                return False
        else:
            return False
    def __hash__(self):
        return hash((self.origin, self.destination, self.beginTime))
    
    def __str__(self) -> str:
        return (self.parentTripId+" origin "+self.origin +" to "+self.destination +" at "+str(self.beginTime))
    
    
    def to_dict(self):
        
        return {"trip_id":self.parentTripId,"origin":self.origin,"destination":self.destination,"begin_time":self.beginTime}

In [ ]:
t1=EfaTrip()

In [21]:
TData=todayTrips.iloc[1]
TData2=todayTrips.iloc[2]
TData3=todayTrips.iloc[3]

t1=EfaTrip(TData["begin_time"],TData["end_time"],TData["origin"],TData["destination"],TData["trip_id"])
t2=EfaTrip(TData2["begin_time"],TData2["end_time"],TData2["origin"],TData2["destination"],TData2["trip_id"])
t3=EfaTrip(TData3["begin_time"],TData3["end_time"],TData3["origin"],TData3["destination"],TData3["trip_id"])

A={t1}
B={t1,t2,t3}

I=A.intersection(B)

for el in I:
    print(el.to_dict())




{'trip_id': 'de:vvs:10001_::1-T0-101-j24', 'origin': 'de:08116:7802:1:9', 'destination': 'de:08111:6115:2:4', 'begin_time': Timestamp('2024-11-06 04:23:00')}


In [37]:
from sollfahrplanreader.vvscurrentdayanalyser import VVSCurrentDayAnalysis

vvsAnanlysis=VVSCurrentDayAnalysis("run1")

todayVVSTrips=vvsAnanlysis.getDeparturesAndArrivalsToday()

c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator
c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator\sollfahrplanreader\vvscurrentdayanalyser.py:24: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  calendar_formatted = pd.read_csv(os.path.join(self.root,"data",self.provider,"gtfs","calendar.txt"), parse_dates=['start_date','end_date'], date_parser=date_parser)
c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator\sollfahrplanreader\vvscurrentdayanalyser.py:26: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  calendar_dates=pd.read_csv(os.path.join(self.root,"data",self.provider,"gtfs","calendar_dates.txt"),parse_dates=['date'], date_parser=date_parser)


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator\sollfahrplanreader\vvscurrentdayanalyser.py:80: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  stopTimes=pd.read_csv(os.path.join(self.parentDirectory,"data",self.provider,"gtfs","stop_times.txt"),dtype=dtype)


number of origins 31298
number of destinations 31298


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator\sollfahrplanreader\vvscurrentdayanalyser.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todaySubset["minute of day"]=todaySubset['timestamp'].dt.hour * 60 + todaySubset['timestamp'].dt.minute


In [32]:
from sollfahrplanreader.nvbwcurrentdayanalyser import NVBWCurrentDayAnalysis
nvbwAnalysis=NVBWCurrentDayAnalysis("run2")

todayNVBWTrips=nvbwAnalysis.getDeparturesAndArrivalsToday()

c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator
c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator\sollfahrplanreader\nvbwcurrentdayanalyser.py:24: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  calendar_formatted = pd.read_csv(os.path.join(self.root,"data",self.provider,"gtfs","calendar.txt"), parse_dates=['start_date','end_date'], date_parser=date_parser)
c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator\sollfahrplanreader\nvbwcurrentdayanalyser.py:26: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  calendar_dates=pd.read_csv(os.path.join(self.root,"data",self.provider,"gtfs","calendar_dates.txt"),parse_dates=['date'], date_parser=date_parser)


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator\sollfahrplanreader\nvbwcurrentdayanalyser.py:80: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  stopTimes=pd.read_csv(os.path.join(self.parentDirectory,"data",self.provider,"gtfs","stop_times.txt"),dtype=dtype)


number of origins 126259
number of destinations 126259


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator\sollfahrplanreader\nvbwcurrentdayanalyser.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todaySubset["minute of day"]=todaySubset['timestamp'].dt.hour * 60 + todaySubset['timestamp'].dt.minute


In [34]:
todayNVBWTrips.head()

,trip_id,begin_time,end_time,agency,route_long_name,number of updates,origin,destination
0,ovf-1-5523-1-1-1-R-0-Special-1-1,2024-11-06 04:45:00,2024-11-06 05:38:00,Default,"Wertheim, ZOB-Würzburg Neue Universität",0,de:09676:1254:2:2,de:08128:13401:0:2
1,ovf-1-5523-1-1-2-R-1-Special-2-2,2024-11-06 06:45:00,2024-11-06 07:40:00,Default,"Wertheim, ZOB-Würzburg Neue Universität",0,de:08128:13401:0:2,de:09663:75:40:112
2,ovf-1-5523-1-1-3-R-2-Special-3-3,2024-11-06 05:40:00,2024-11-06 06:35:00,Default,"Wertheim, ZOB-Würzburg Neue Universität",0,de:08128:13401:0:2,de:09663:274:0:12
3,ovf-1-5523-1-1-3-R-2-Special-7-4,2024-11-06 10:40:00,2024-11-06 11:36:00,Default,"Wertheim, ZOB-Würzburg Neue Universität",0,de:08128:13401:0:2,de:09663:274:0:12
4,ovf-1-5523-1-1-3-R-2-Special-8-5,2024-11-06 12:40:00,2024-11-06 13:41:00,Default,"Wertheim, ZOB-Würzburg Neue Universität",0,de:08128:13401:0:2,de:09663:274:0:12


In [38]:
class EfaTrip:
    def __init__(self,beginTime,endTime,origin,destination,parentTripId) -> None:
        self.beginTime=beginTime
        self.endTime=endTime
        self.origin=origin
        self.destination=destination
        self.parentTripId=parentTripId
        
    def __eq__(self, other: object) -> bool:
        if isinstance(other,EfaTrip):
            print("instance test ")
            if((self.beginTime==other.beginTime) and (self.endTime==other.endTime) and (self.origin==other.origin) and (self.destination==other.destination)):
                return True
            else:
                return False
        else:
            return False
    def __hash__(self):
        return hash((self.origin, self.destination, self.beginTime))
    
    def __str__(self) -> str:
        return (self.parentTripId+" origin "+self.origin +" to "+self.destination +" at "+str(self.beginTime))
    
    
    def to_dict(self):
        
        return {"trip_id":self.parentTripId,"origin":self.origin,"destination":self.destination,"begin_time":self.beginTime}
    
    
def mapRowToEfaTrip(row):
    TData=row
    trip=EfaTrip(TData["begin_time"],TData["end_time"],TData["origin"],TData["destination"],TData["trip_id"])  
    return trip  


VVStrips = todayVVSTrips.apply(lambda row: mapRowToEfaTrip(row), axis=1).tolist()


In [40]:
NVBWTrips=todayNVBWTrips.apply(lambda row: mapRowToEfaTrip(row), axis=1).tolist()

In [41]:
VVSTripSet=set(VVStrips)
NVBWTripSet=set(NVBWTrips)

instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance t

In [44]:
exclusiveVVS=VVSTripSet.difference(NVBWTripSet)

instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance test 
instance t

In [46]:
for el in exclusiveVVS:
    print(el.to_dict())

{'trip_id': 'de:vvs:11005_::1-T0-111-j24', 'origin': 'de:09776:1592:1:1', 'destination': 'de:08111:6115:5:9', 'begin_time': Timestamp('2024-11-06 05:28:00')}
{'trip_id': 'de:vvs:10060_::1-T0-93-j24', 'origin': 'de:08115:7100:1:5', 'destination': 'de:08115:7302:1:2', 'begin_time': Timestamp('2024-11-07 00:34:00')}
{'trip_id': 'de:vvs:10002_::1-T0-64-j24', 'origin': 'de:08116:1905:1:2', 'destination': 'de:08119:7703:1:1', 'begin_time': Timestamp('2024-11-06 09:33:00')}
{'trip_id': 'de:vvs:10002_::1-T0-30-j24', 'origin': 'de:08119:7703:1:14', 'destination': 'de:08116:1905:1:1', 'begin_time': Timestamp('2024-11-06 13:48:00')}
{'trip_id': 'de:vvs:10002_::1-T0-52-j24', 'origin': 'de:08116:1905:1:1', 'destination': 'de:08119:7703:1:1', 'begin_time': Timestamp('2024-11-06 05:03:00')}
{'trip_id': 'de:vgc-online:15759_: :3-T0-28-j24', 'origin': 'de:08235:7330:0:4', 'destination': 'de:08235:7000:0:4', 'begin_time': Timestamp('2024-11-06 16:27:00')}
{'trip_id': 'de:vvs:11016_::66-T0-76-j24', 'orig

In [1]:
from sollfahrplanreader.vvscurrentdayanalyser import VVSCurrentDayAnalysis
from sollfahrplanreader.nvbwcurrentdayanalyser import NVBWCurrentDayAnalysis
from sollfahrplanreader.efaTrip import EfaTrip



def mapRowToEfaTrip(row):
    TData=row
    trip=EfaTrip(TData["begin_time"],TData["end_time"],TData["origin"],TData["destination"],TData["trip_id"])  
    return trip  






nvbwAnalysis=NVBWCurrentDayAnalysis("run2")

todayNVBWTrips=nvbwAnalysis.getDeparturesAndArrivalsToday()

vvsAnanlysis=VVSCurrentDayAnalysis("run1")

todayVVSTrips=vvsAnanlysis.getDeparturesAndArrivalsToday()



VVStrips = todayVVSTrips.apply(lambda row: mapRowToEfaTrip(row), axis=1).tolist()
NVBWTrips=todayNVBWTrips.apply(lambda row: mapRowToEfaTrip(row), axis=1).tolist()

VVSTripSet=set(VVStrips)
NVBWTripSet=set(NVBWTrips)
exclusiveVVS=VVSTripSet.difference(NVBWTripSet)

exclusiveTripsDict=[]

for el in exclusiveVVS:
    exclusiveTripsDict.append(el.to_dict())
    print(el.to_dict())

c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator
c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator\sollfahrplanreader\nvbwcurrentdayanalyser.py:24: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  calendar_formatted = pd.read_csv(os.path.join(self.root,"data",self.provider,"gtfs","calendar.txt"), parse_dates=['start_date','end_date'], date_parser=date_parser)
c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator\sollfahrplanreader\nvbwcurrentdayanalyser.py:26: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  calendar_dates=pd.read_csv(os.path.join(self.root,"data",self.provider,"gtfs","calendar_dates.txt"),parse_dates=['date'], date_parser=date_parser)


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator\sollfahrplanreader\nvbwcurrentdayanalyser.py:80: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  stopTimes=pd.read_csv(os.path.join(self.parentDirectory,"data",self.provider,"gtfs","stop_times.txt"),dtype=dtype)


number of origins 126259
number of destinations 126259


...

c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator\sollfahrplanreader\vvscurrentdayanalyser.py:80: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  stopTimes=pd.read_csv(os.path.join(self.parentDirectory,"data",self.provider,"gtfs","stop_times.txt"),dtype=dtype)


number of origins 31298
number of destinations 31298


c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator\sollfahrplanreader\vvscurrentdayanalyser.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todaySubset["minute of day"]=todaySubset['timestamp'].dt.hour * 60 + todaySubset['timestamp'].dt.minute


{'trip_id': 'de:naldo:bd9146d427364d8780b3fafc3fc269a1::1-T0-30-j24', 'origin': 'de:08416:1004:0:4', 'destination': 'de:08115:4510:3:3', 'begin_time': Timestamp('2024-11-06 13:01:00')}
{'trip_id': 'de:vvs:10002_::1-T0-166-j24', 'origin': 'de:08116:1905:1:1', 'destination': 'de:08119:7703:1:14', 'begin_time': Timestamp('2024-11-06 10:18:00')}
{'trip_id': 'de:vvs:11018_::1-T0-27-j24', 'origin': 'de:08225:8200:2:3', 'destination': 'de:08125:519:4:5', 'begin_time': Timestamp('2024-11-06 06:14:00')}
{'trip_id': 'de:vvs:10006_::1-T0-86-j24', 'origin': 'de:08111:6052:1:2', 'destination': 'de:08115:1303:1:2', 'begin_time': Timestamp('2024-11-06 15:57:00')}
{'trip_id': 'de:vvs:11016_::66-T0-42-j24', 'origin': 'de:08421:1008:32:3', 'destination': 'de:08111:6115:5:9', 'begin_time': Timestamp('2024-11-06 14:36:00')}
{'trip_id': 'de:vvs:31774a::1-T0-1-j24', 'origin': 'de:08115:4860:0:3', 'destination': 'de:08115:4509:0:4', 'begin_time': Timestamp('2024-11-06 13:00:00')}
{'trip_id': 'de:naldo:7e75e6

In [6]:
import pandas as pd
import os
from getrootdirectory import getRootDirectory

exclusiveVVSDf=pd.DataFrame.from_dict(exclusiveTripsDict)

exclusiveVVSDf.head()

exclusiveVVSDf.to_csv(os.path.join(getRootDirectory(),"data","exclusiveVVS.csv"),index=False)

c:\Users\CHO\Documents\GTFSRTAnalysis\StadtnaviHerrenberg\comparator


In [9]:
len(NVBWTripSet)
len(VVSTripSet)
len(exclusiveVVS)

1024